In [ ]:
!python --version

Python 3.10.6


In [ ]:
!pip install emot
!pip install dateutil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.1 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement dateutil (from versions: none)
ERROR: No matching distribution found for dateutil


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import re
from itertools import chain
from dateutil.parser import parse
import time
from datetime import datetime, timedelta
from collections import OrderedDict, defaultdict, Counter
from nltk.tokenize import TweetTokenizer
from emot.emo_unicode import UNICODE_EMOJI

# Setup

## PySpark

In [ ]:
!pwd
!wget https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar -xzf spark-3.4.1-bin-hadoop3.tgz
!ls /content/spark-3.4.1-bin-hadoop3
!pip install findspark
!pip install pyspark

/content
--2023-07-20 23:51:57--  https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388341449 (370M) [application/x-gzip]
Saving to: ‘spark-3.4.1-bin-hadoop3.tgz’

spark-3.4.1-bin-had 100%[===================>] 370.35M   236MB/s    in 1.6s    

2023-07-20 23:51:58 (236 MB/s) - ‘spark-3.4.1-bin-hadoop3.tgz’ saved [388341449/388341449]

bin   data	jars	    LICENSE   NOTICE  R		 RELEASE  yarn
conf  examples	kubernetes  licenses  python  README.md  sbin
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=1b89133f9994da236198e6539feb795d74cf319cfd9114e275c36534dcc56950
  Stored in directory: /ro

## Kafka

In [ ]:
!pip install kafka-python
!wget https://downloads.apache.org/kafka/3.5.0/kafka_2.12-3.5.0.tgz
!tar -xzf kafka_2.12-3.5.0.tgz
!./kafka_2.12-3.5.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.12-3.5.0/config/zookeeper.properties
!./kafka_2.12-3.5.0/bin/kafka-server-start.sh -daemon ./kafka_2.12-3.5.0/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 5.5 MB/s eta 0:00:00
--2023-07-20 23:52:44--  https://downloads.apache.org/kafka/3.5.0/kafka_2.12-3.5.0.tgz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.95.219, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 107000763 (102M) [application/x-gzip]
Saving to: ‘kafka_2.12-3.5.0.tgz’

kafka_2.12-3.5.0.tg 100%[===================>] 102.04M  23.2MB/s    in 5.2s    

2023-07-20 23:52:49 (19.5 MB/s) - ‘kafka_2.12-3.5.0.tgz’ saved [107000763/107000763]

Waiting for 10 secs until kafka and zookeeper services are up and running


In [ ]:
!ps -ef | grep kafka

root        1935       1 13 23:52 ?        00:00:01 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.12-3.5.0/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.12-3.5.0/bin/../logs -Dlog4j.configuration=file:./kafka_2.12-3.5.0/bin/../config/log4j.properties -cp /content/kafka_2.12-3.5.0/bin/../libs/activation-1.1.1.jar:/content/kafka_2.12-3.5.0/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.12-3.5.0/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.12-3.5.0/bin/../libs/audience-annotations-0.13.0.jar:/content/kafka_2.12-3.5.0/bin/../libs/commons-cli-1.4.jar:/content/kafka_2.12-3.5.0/bin/../libs/commons-lang3-3.8.1.jar:/content/kafka_2.1

## Create the kafka topics


1.   Traffic: partitions = 1, replication-factor=1
2.   Weather: partitions = 1, replication-factor=1
3. News: partitions = 1, replication-factor=1

In [ ]:
!./kafka_2.12-3.5.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic Traffic
!./kafka_2.12-3.5.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic Weather
!./kafka_2.12-3.5.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic News

Created topic Traffic.
Created topic Weather.
Created topic News.


> Describe the topic for details on the configuration



In [ ]:
!./kafka_2.12-3.5.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic Traffic
!./kafka_2.12-3.5.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic Weather
!./kafka_2.12-3.5.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic News

Topic: Traffic	TopicId: M4NjkOmYSFuLnlv4j7QTcw	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: Traffic	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
Topic: Weather	TopicId: 1ByYwfshRPmG5xu5BTPQEg	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: Weather	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
Topic: News	TopicId: _NGbQptsStKPhCo7HKWLHw	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: News	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


In [ ]:
import os
import findspark
from pyspark.sql import SparkSession

os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"
findspark.init()

scala_version = '2.12'  # your scala version
spark_version = '3.4.1' # your spark version
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:2.12-3.5.0' #your kafka version
]
spark = SparkSession.builder.master("local").appName("kafka-example").config("spark.jars.packages", ",".join(packages)).getOrCreate()
spark

### Load Station Info

In [ ]:
station_info = pd.read_csv('/content/drive/MyDrive/Phân tích dữ liệu lớn - DS200.N21/Data/Traffic/final_station_info.csv')

In [ ]:
suburb_list = list(set(station_info['suburb']))
road_name = list(set(station_info['road_name']))
intersaction = list(set(station_info['intersection']))

# Data

### Load Station Info

In [ ]:
station_info = pd.read_csv('/content/drive/MyDrive/Phân tích dữ liệu lớn - DS200.N21/Data/Traffic/final_station_info.csv')

In [ ]:
suburb_list = list(set(station_info['suburb']))
road_name = list(set(station_info['road_name']))
intersaction = list(set(station_info['intersection']))



## Traffic



In [ ]:
# Dataframe to simulate real-time data flow
DATA_PATH = '/content/drive/MyDrive/Phân tích dữ liệu lớn - DS200.N21/Data/Traffic/raw_traffic.csv'
raw_traffic = pd.read_csv(DATA_PATH)
raw_traffic

,datetime,01004,02384,02413,03022,100001,10011,19065,22001,22043,...,74229,9658,9827,9834-PR,9836-PR,9849,F3FWY001,T0085,T0231,T0345
0,2019-07-07 00:00:00,880.0,590.0,140.0,286.0,NaN,1428.0,1544.0,2222.0,96.0,...,371.0,254.0,200.0,256.0,1058.0,272.0,688.0,36.0,400.0,164.0
1,2019-07-07 01:00:00,562.0,411.0,90.0,215.0,NaN,1054.0,1170.0,1438.0,54.0,...,174.0,158.0,135.0,194.0,751.0,173.0,347.0,18.0,297.0,89.0
2,2019-07-07 02:00:00,433.0,315.0,77.0,146.0,NaN,941.0,948.0,1020.0,31.0,...,124.0,95.0,100.0,95.0,633.0,97.0,265.0,12.0,255.0,48.0
3,2019-07-07 03:00:00,341.0,265.0,73.0,116.0,NaN,639.0,747.0,922.0,18.0,...,93.0,90.0,51.0,91.0,488.0,73.0,276.0,9.0,196.0,28.0
4,2019-07-07 04:00:00,296.0,206.0,100.0,100.0,NaN,335.0,623.0,710.0,16.0,...,94.0,96.0,32.0,54.0,394.0,73.0,446.0,8.0,170.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26299,2022-07-06 19:00:00,1387.0,804.0,323.0,455.0,1591.0,1486.0,2572.0,3746.0,509.0,...,1006.0,NaN,466.0,390.0,968.0,NaN,678.0,NaN,NaN,NaN
26300,2022-07-06 20:00:00,1013.0,659.0,228.0,396.0,1241.0,1210.0,2327.0,2958.0,323.0,...,757.0,NaN,402.0,299.0,801.0,NaN,551.0,NaN,NaN,NaN
26301,2022-07-06 21:00:00,1100.0,592.0,231.0,363.0,1081.0,1110.0,2169.0,3006.0,268.0,...,756.0,NaN,390.0,226.0,593.0,NaN,401.0,NaN,NaN,NaN
26302,2022-07-06 22:00:00,1004.0,483.0,160.0,280.0,1007.0,777.0,1827.0,2218.0,162.0,...,555.0,NaN,255.0,97.0,474.0,NaN,345.0,NaN,NaN,NaN


In [ ]:
def split_realtime_data(day, df):
  total_day = int(len(df) / 24)
  historical_day = total_day - day
  historical_df = df[:day*24]
  realtime_df = df[historical_day*24:]
  return historical_df, realtime_df

In [ ]:
historical_raw_traffic, realtime_raw_traffic = split_realtime_data(9, raw_traffic)



##  News



In [ ]:
# Dataframe to simulate real-time news data
DATA_PATH = '/content/drive/MyDrive/Phân tích dữ liệu lớn - DS200.N21/Data/News/Total_related_news.csv'
raw_news = pd.read_csv(DATA_PATH, index_col = 0)
raw_news

,post_id,text,time,likes,comments,shares,post_url
0,364669825774779,ROADS AFFECTED BY FLOODING - UPDATE ON THURSDA...,06-07-2022 20:28:02,289,118,85,https://facebook.com/livetrafficnsw/posts/3646...
1,364070192501409,ROADS AFFECTED BY FLOODING - UPDATE ON WEDNESD...,05-07-2022 21:06:12,316,113,76,https://facebook.com/livetrafficnsw/posts/3640...
2,363611919213903,ROADS AFFECTED BY FLOODING - UPDATE ON TUESDAY...,05-07-2022 04:36:59,520,69,171,https://facebook.com/livetrafficnsw/posts/3636...
3,363421322566296,LATEST ON ROADS AFFECTED BY FLOODING AT 12PM:\...,04-07-2022 20:13:46,409,90,134,https://facebook.com/livetrafficnsw/posts/3634...
4,362815415960220,LATEST ON FLOOD AFFECTED ROADS ON MONDAY 4 JUL...,03-07-2022 20:32:15,272,42,80,https://facebook.com/livetrafficnsw/posts/3628...
...,...,...,...,...,...,...,...
474,5268775016527496,HAMMONDVILLE - FINAL UPDATE 7:10AM: All eastbo...,12-04-2021 20:34:43,32,14,5,https://facebook.com/livetrafficnsw/posts/5268...
475,4813260578745611,RANDWICK - FINAL UPDATE AT 6:55PM: All eastbou...,05-01-2021 06:41:18,22,0,4,https://facebook.com/livetrafficnsw/posts/4813...
476,4453749754696697,KENSINGTON - FINAL UPDATE AS OF 2:30AM: .South...,12-10-2020 08:29:36,11,2,4,https://facebook.com/livetrafficnsw/posts/4453...
477,4136198926451783,KYEEMAGH: The M5 East Motorway is closed westb...,11-08-2020 15:23:45,25,2,2,https://facebook.com/livetrafficnsw/posts/4136...


In [ ]:
start_date = '2022-06-28'
end_date = '2022-07-06'

def split_realtime_news(start_date, end_date, df):
  df['date'] = pd.to_datetime(df['time'], format = "%d-%m-%Y %H:%M:%S").dt.strftime('%Y-%m-%d')
  news_by_date = df[df['date'].isin(pd.date_range(start_date,end_date).strftime("%Y-%m-%d"))]
  df = df.drop(['date'], axis = 1)
  news_by_date = news_by_date.sort_values(by='time').drop(['date'], axis =1).reset_index(drop=True)
  return news_by_date

In [ ]:
realtime_raw_news = split_realtime_news(start_date, end_date, raw_news)
realtime_raw_news

,post_id,text,time,likes,comments,shares,post_url
0,361677882740640,ROYAL NATIONAL PARK:\nAudley Weir is closed be...,02-07-2022 03:12:30,190,7,12,https://facebook.com/livetrafficnsw/posts/3616...
1,361798246061937,FLOODED ROADS IN SYDNEY - UPDATE AT 8:30PM:\n-...,02-07-2022 07:52:25,514,60,139,https://facebook.com/livetrafficnsw/posts/3617...
2,361934472714981,MILPERRA - UPDATE AT 11:50PM: Due to further f...,02-07-2022 12:58:11,230,29,32,https://facebook.com/livetrafficnsw/posts/3619...
3,361967616045000,GEORGES HALL - UPDATE 3:45AM: Henry Lawson Dri...,02-07-2022 14:03:54,246,25,32,https://facebook.com/livetrafficnsw/posts/3619...
4,362252232683205,LATEST ON FLOOD AFFECTED ROADS IN SYDNEY AS OF...,02-07-2022 23:54:57,608,70,107,https://facebook.com/livetrafficnsw/posts/3622...
5,362815415960220,LATEST ON FLOOD AFFECTED ROADS ON MONDAY 4 JUL...,03-07-2022 20:32:15,272,42,80,https://facebook.com/livetrafficnsw/posts/3628...
6,363421322566296,LATEST ON ROADS AFFECTED BY FLOODING AT 12PM:\...,04-07-2022 20:13:46,409,90,134,https://facebook.com/livetrafficnsw/posts/3634...
7,363611919213903,ROADS AFFECTED BY FLOODING - UPDATE ON TUESDAY...,05-07-2022 04:36:59,520,69,171,https://facebook.com/livetrafficnsw/posts/3636...
8,364070192501409,ROADS AFFECTED BY FLOODING - UPDATE ON WEDNESD...,05-07-2022 21:06:12,316,113,76,https://facebook.com/livetrafficnsw/posts/3640...
9,364598869115208,HALFWAY CREEK: FINAL UPDATE AT 5:00PM: All lan...,06-07-2022 17:38:37,941,289,283,https://facebook.com/livetrafficnsw/posts/3645...


In [ ]:
station_list = list(realtime_raw_traffic.columns)
station_list.remove('datetime')

In [ ]:
print(station_list)

['01004', '02384', '02413', '03022', '100001', '10011', '19065', '22001', '22043', '24014', '24026', '28022', '29005', '32022', '33013', '33014', '33023', '36002', '40041', '42001', '50232', '50240', '51235', '53003', '55049', '57025', '66249', '68025', '7104-PR', '7112', '7119-PR', '7120', '7121', '7123-PR', '7126', '7132', '7139', '7149', '7150', '7153', '7159', '7161', '7162', '7163', '7164', '7168', '7179', '7184', '72027', '7251', '7270', '7272', '7274', '7275', '74228', '74229', '9658', '9827', '9834-PR', '9836-PR', '9849', 'F3FWY001', 'T0085', 'T0231', 'T0345']




##  Weather


In [ ]:
weather_list = []
for station in station_list:
  name = 'station' + str(station)
  name = pd.read_csv('/content/drive/MyDrive/Phân tích dữ liệu lớn - DS200.N21/Data/Weather/raw/{}.csv'.format(station), index_col = 0)
  name.insert(0, 'station_id', station)
  historical_raw_weather, realtime_raw_weather = split_realtime_data(9, name)
  weather_list.append(realtime_raw_weather)

realtime_raw_weather = pd.concat(weather_list).sort_values(by = 'datetime').groupby(['datetime'], group_keys=True).apply(lambda x: x.to_json(orient="records"))
realtime_raw_weather = pd.DataFrame(realtime_raw_weather)

In [53]:
weather_list[0].set_index('datetime')

,station_id,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation,rain,snowfall,weathercode,pressure_msl,...,winddirection_100m,windgusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm
datetime,,,,,,,,,,,,,,,,,,,,,
2022-06-28 00:00:00,01004,12.6,72,7.7,10.3,0.0,0.0,0.0,0,1031.4,...,232,28.4,12.7,12.5,14.4,17.0,0.172,0.173,0.194,0.271
2022-06-28 01:00:00,01004,13.5,70,8.1,11.3,0.0,0.0,0.0,1,1030.9,...,250,29.5,13.7,12.6,14.4,17.0,0.172,0.173,0.194,0.270
2022-06-28 02:00:00,01004,14.3,68,8.4,11.9,0.0,0.0,0.0,1,1030.3,...,263,29.9,14.5,12.8,14.4,17.0,0.172,0.172,0.194,0.270
2022-06-28 03:00:00,01004,14.7,67,8.6,12.3,0.0,0.0,0.0,3,1029.8,...,268,30.6,15.2,13.0,14.4,17.0,0.172,0.172,0.194,0.270
2022-06-28 04:00:00,01004,14.7,67,8.6,12.5,0.0,0.0,0.0,3,1029.4,...,269,29.5,15.0,13.1,14.4,17.0,0.172,0.172,0.194,0.270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-06 19:00:00,01004,14.6,92,13.4,13.3,0.1,0.1,0.0,51,1010.8,...,233,33.5,14.2,14.4,14.7,16.5,0.342,0.329,0.339,0.300
2022-07-06 20:00:00,01004,14.6,90,13.0,13.2,0.1,0.1,0.0,51,1011.1,...,228,33.5,14.1,14.4,14.7,16.5,0.340,0.331,0.339,0.301
2022-07-06 21:00:00,01004,14.4,89,12.6,12.8,0.1,0.1,0.0,51,1011.3,...,222,33.1,13.9,14.4,14.7,16.5,0.337,0.331,0.339,0.301


# Preprocessing Function

### Function Preprocessing Traffic

In [ ]:
def preprocessing_traffic(df):
  df = df.fillna(0)
  df = df.set_index('datetime')
  return df

### Preprocessing News Function

1.   Extract news information


In [ ]:
emoji_list = list(UNICODE_EMOJI)

def split_emoji_text(text):
    result = ''.join((' {} '.format(el) if el in emoji_list else el for el in text))
    return result

def regex_or(*items):
    return '(?:' + '|'.join(items) + ')'

delimeter1 = r"[\*]{3,}"
delimeter2 = r"[\+]+"
delimeter3 = r"Live Traffic NSW"
delimeter4 = r"NSW"
regex = regex_or(delimeter1, delimeter2, delimeter3, delimeter4)
regex_suburb = r"\b(?:{})".format("|".join(suburb_list))
regex_road = r"\b(?:{})".format("|".join(road_name))
regex_intersection = r"\b(?:{})".format("|".join(intersaction))
regex_date = r"\b([A-Za-z]{0,}[\.\,\s]{0,}[A-Za-z]{1,}[\.\s]{1,}[0-9]{1,2}[\,\s]{0,}[0-9]{4})\b"
regex_time = r"((1[0-2]|0?[1-9]):([0-5][0-9])((\\s| )?[AaPp][Mm]))"

In [ ]:
import datetime

def return_value(list):
  if (len(list)==0):
    return np.nan
  return list

def return_date(date):
  return parse(str(date), fuzzy=True).strftime("%d-%m-%Y")

def return_time(time):
  return parse(str(time), fuzzy=True).strftime("%H:%M:%S")

def round_time(dt=None, date_delta=datetime.timedelta(minutes=1), to='average'):
    """
    Round a datetime object to a multiple of a timedelta
    dt : datetime.datetime object, default now.
    dateDelta : timedelta object, we round to a multiple of this, default 1 minute.
    from:  http://stackoverflow.com/questions/3463930/how-to-round-the-minute-of-a-datetime-object-python
    """
    round_to = date_delta.total_seconds()
    if dt is None:
        dt = datetime.now()
    seconds = (dt - dt.min).seconds

    if seconds % round_to == 0 and dt.microsecond == 0:
        rounding = (seconds + round_to / 2) // round_to * round_to
    else:
        if to == 'up':
            # // is a floor division, not a comment on following line (like in javascript):
            rounding = (seconds + dt.microsecond/1000000 + round_to) // round_to * round_to
        elif to == 'down':
            rounding = seconds // round_to * round_to
        else:
            rounding = (seconds + round_to / 2) // round_to * round_to

    return dt + datetime.timedelta(0, rounding - seconds, - dt.microsecond)

In [ ]:
def split_news_by_update(realtime_news, df):
  for i in range(len(realtime_news)):
    id = realtime_news['post_id'][i]
    posted_time = realtime_news['time'][i]
    text = realtime_news['text'][i]
    text = split_emoji_text(text)
    text_split_by_update = re.split(regex, text)
    for j in range(len(text_split_by_update)):
      event = text_split_by_update[j]
      new_row = pd.DataFrame({'post_id': id, 'text':event, 'posted_time': posted_time, 'polarity': 0} , index=[0])
      df = pd.concat([new_row, df.loc[:]]).reset_index(drop=True)
  return df

def extract_event_time(event_df):
  for i in range(len(event_df)):
    text = event_df['text'][i]
    posted_time = event_df['posted_time'][i]
    date = re.findall(regex_date, text)
    time = re.findall(regex_time, text)

    if (len(time) != 0):
      event_df['event_time'][i] = return_time(time[0][0])
    else:
      event_df['event_time'][i] = return_time(posted_time)

    if (len(date) != 0):
      event_df['event_date'][i] = return_date(date)
    else:
      event_df['event_date'][i] = return_date(posted_time)

    event_df['event_datetime'][i] = event_df['event_date'][i] + ' ' + event_df['event_time'][i]
    event_df['event_datetime'][i] = pd.to_datetime(event_df['event_datetime'][i], format='%d-%m-%Y %H:%M:%S')
    event_df['event_datetime'][i] = round_time(pd.Timestamp.to_pydatetime(event_df['event_datetime'][i]), date_delta=datetime.timedelta(minutes=60), to='up')
    event_df['event_datetime'][i] = event_df['event_datetime'][i].strftime('%m-%d-%Y %H:%M:%S')
  return event_df

def extract_event_location(event_df):
  for i in range(len(event_df)):
    text = event_df['text'][i]

    suburb = list(set(re.findall(regex_suburb, text)))
    event_df['count_suburb'][i] = len(suburb)
    event_df['suburb'][i] = return_value(suburb)

    road = list(set(re.findall(regex_road, text)))
    event_df['count_road_name'][i] = len(road)
    event_df['road_name'][i] = return_value(road)

    intersection = list(set(re.findall(regex_intersection, text)))
    event_df['count_intersection'][i] = len(intersection)
    event_df['intersection'][i] = return_value(intersection)

  event_df = event_df.dropna(subset=['suburb','road_name', 'intersection'], thresh=1).reset_index(drop=True)
  return event_df

def extract_related_station(event_df):
  related_station = []
  types = ['suburb', 'road_name', 'intersection']
  for i in range(len(event_df)):
    for type in types:
      for j in range(event_df['count_' + type][i]):
        if (type == 'intersection'):
          related_station.append(list(set(station_info.loc[(station_info[type] == event_df[type][i][j]) & (station_info['distance_to_intersection'] < 40)]['station_id'])))
        else:
          related_station.append(list(set(station_info.loc[station_info[type] == (event_df[type][i][j]) ]['station_id'])))
    event_df['related_station'][i] = list(set(chain.from_iterable(related_station)))
  return event_df



2.   Calculate news polarity



In [ ]:
wordDict = defaultdict(float)

with open('/content/drive/MyDrive/Phân tích dữ liệu lớn - DS200.N21/Data/News/Emoticon-AFFLEX-NEGLEX-unigrams.txt', 'r') as f:
    for row in f.readlines():
        row = row.split()
        wordDict[row[0]] = float(row[1])

In [ ]:
tt = TweetTokenizer()

def polarity(x):
    score = wordDict[x]
    if score > 0:
        return 'positive'
    if score < 0:
        return 'negative'
    else:
        return 'none'

def count_tokens_with_polarity(string, tokenizer):

    scorelist = []
    for token in tt.tokenize(string):
        token = token.lower()
        score = polarity(token)
        scorelist.append(score)

    return dict(Counter(scorelist))

def polarity_sum(string, tokenizer):

    negList = []
    posList = []

    for token in tt.tokenize(string):
        token = token.lower()
        if polarity(token) == 'positive':
            posList.append(wordDict[token])
        elif polarity(token) == 'negative':
            negList.append(abs(wordDict[token]))

    return {'pos_sum' : sum(posList), 'neg_sum' : sum(negList), 'sentiment_score': round((sum(posList) - sum(negList)), 4)}

def max_token(string, tokenizer):
    negList = []
    posList = []

    for token in tt.tokenize(string):
        token = token.lower()
        if polarity(token) == 'positive':
            posList.append(wordDict[token])
        elif polarity(token) == 'negative':
            negList.append(wordDict[token])
    try:
        pos_max = max(posList)
    except ValueError:
        pos_max = 0
    try:
        neg_max = min(negList)
    except ValueError:
        neg_max = 0

    return {'pos_max' : pos_max, 'neg_max' : neg_max}

def last_token(string, tokenizer):

    negList = []
    posList = []
    for token in reversed(tt.tokenize(string)):
        token = token.lower()
        if polarity(token) == 'positive' or polarity(token) == 'negative':
            return {'last_polarity' : wordDict[token]}
        else:
            continue

    return {'last_polarity' : 0}

def all_feats_dict(string, tokenizer):
    ct = count_tokens_with_polarity(string, tokenizer)
    pol = polarity_sum(string, tokenizer)
    max_tkn = max_token(string, tokenizer)
    last = last_token(string, tokenizer)

    complete = dict()
    for dictionary in [ct, pol, max_tkn, last]:
        complete.update(dictionary)
    return complete

def calculate_news_polarity(event_df):
    df = event_df[['post_id', 'text', 'polarity']]
    tweets = df['text']
    emotionCounts = [all_feats_dict(tweet, tt) for tweet in tweets]
    emotion_df = pd.DataFrame(emotionCounts, index=tweets.index)
    emotion_df = emotion_df.fillna(0)
    event_df['polarity'] = emotion_df['sentiment_score']
    return event_df


3. Transform news polarity into station df

In [ ]:
start_time = '2022-06-28 00:00:00'
end_time = '2023-07-06 23:00:00'

def create_empty_df(start, end):
  datetime = pd.date_range(start = start, end = end, freq = 'H').strftime("%d-%m-%Y %H:%M:%S")
  columns = list(station_info['station_id'])
  columns.insert(0,'datetime')
  news_df = pd.DataFrame(data = [[float(0)]*len(columns)]*len(datetime), columns = columns)
  news_df['datetime'] = datetime
  return news_df


def transform_news_to_df(news_df, empty_df):
  for i in range(len(empty_df)):
    station_dic = {}
    date = empty_df['datetime'][i]
    for j in range(len(news_df)):
      if (news_df['event_datetime'][j] == date):
        for station in news_df['related_station'][j]:
          id = news_df['event_datetime'][j]
          if station not in station_dic:
                station_dic[station] = {}
          if id not in station_dic[station]:
                station_dic[station][id]= 1
          else:
                station_dic[station][id] += 1
          empty_df[station][i] += news_df['polarity'][j]
    for station in empty_df.columns:
      if station in station_dic:
          empty_df[station][i] = empty_df[station][i] /station_dic[station][date]
  return empty_df

In [ ]:
def preprocessing_news(input, start_time, end_time):
  output = pd.DataFrame(columns = ['post_id', 'text', 'posted_time', 'event_date', 'event_time', 'event_datetime','polarity','suburb', 'road_name', 'intersection','count_suburb', 'count_road_name', 'count_intersection', 'related_station'])
  df = split_news_by_update(input, output)
  df = extract_event_time(df)
  df = extract_event_location(df)
  df = extract_related_station(df)
  df = calculate_news_polarity(df)
  empty_df = create_empty_df(start_time, end_time)
  news_df = transform_news_to_df(df, empty_df)
  news_df = news_df.set_index('datetime')
  return news_df

In [ ]:
#event_df = pd.DataFrame(columns = ['post_id', 'text', 'posted_time', 'event_date', 'event_time', 'event_datetime','polarity','suburb', 'road_name', 'intersection','count_suburb', 'count_road_name', 'count_intersection', 'related_station'])

#df = preprocessing_news(realtime_news, event_df, start_time, end_time)

### Preprocessing Weather Function

In [ ]:
def convert_dict_to_df(data):
  dict =  json.loads(data)
  return pd.DataFrame.from_dict(dict)

def group_weather_by_station(df):
  x = []
  for i in range(len(df)):
    station = convert_dict_to_df(df[0][i])
    x.append(station)
  return x

def preprocessing_weather(df):
  drop_features = ['rain', 'snowfall', 'weathercode']
  weather_list = []
  for i in range(len(df)):
    time = df[i]
    time = time.drop(drop_features, axis = 1).set_index('datetime')
    weather_list.append(time)

  realtime_raw_weather = pd.concat(weather_list).sort_values(by = 'datetime').groupby(['station_id'], group_keys=True).apply(lambda x: x.to_json(orient="records"))

  realtime_raw_weather = pd.DataFrame(realtime_raw_weather).reset_index()
  data = group_weather_by_station(realtime_raw_weather)
  return data

## Preproessing kafka df

In [ ]:
def convert_dfdict_to_df(df, value):
  traffic = []
  for i in range(len(df)):
    d = json.loads(df[value][i])
    traffic.append(d)
  traffic_df = pd.DataFrame.from_dict(traffic)
  return traffic_df

In [ ]:
def convert_strfdict_to_df(df, value):
  weather = []
  for i in range(len(df)):
    k = json.loads(df[value][i])[str(0)]
    v = json.loads(k)
    weather_df = pd.DataFrame.from_dict(v)
    weather.append(weather_df)

  return weather

In [ ]:
def preprocessing_realtime_data(traffic_kafka, weather_kafka, news_kafka):
  traffic = convert_dfdict_to_df(traffic_kafka, value = 'value')
  news = convert_dfdict_to_df(news_kafka, value = 'value')
  weather = convert_strfdict_to_df(weather_kafka, 'value')
  #Preprocessing
  start_date = '2022-06-28'
  end_date = '2022-07-06'
  traffic = preprocessing_traffic(traffic)
  news = preprocessing_news(news, start_time, end_time)
  weather = preprocessing_weather(weather)
  return traffic, news, weather

def extract_reatime_data(traffic_df, news_df, weather_df):
  X = []

  for station in traffic_df.columns:
    timestep = []
    traffic = traffic_df[station]
    new_lst = news_df[station]
    for j in range(len(weather_df)):
      if (station == list(set(weather_df[j]['station_id']))[0]):
        weather = weather_df[j]
        weather = weather.drop(['station_id'], axis = 1)
    for i in range(len(traffic)):
      features = []
      features.append(traffic[i])
      features.append(new_lst[i])
      weather_lst = list(weather.iloc[i, :])
      for value in weather_lst:
        features.append(value)
      timestep.append(features)
    X.append(timestep)
  X = np.array(X)
  X_reshaped= X.reshape(X.shape[0], -1)
  return X_reshaped

# Producer

In [ ]:
# Constants
from kafka import KafkaProducer
from json import dumps
import json

KAFKA_BOOTSTRAP_SERVERS = 'localhost:9092'
KAFKA_TOPIC_NAME1 = 'Traffic'
KAFKA_TOPIC_NAME2 = 'News'
KAFKA_TOPIC_NAME3 = 'Weather'

In [ ]:
def serializer(data):
    return json.dumps(data).encode('utf-8')

In [ ]:
# Producer object
producer = KafkaProducer(
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    value_serializer=serializer
)

In [ ]:
def error_callback(exc):
      raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))

def write_to_kafka(topic_name, value):
      count=0
      #producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
      messages = value.to_dict('records')
      for message in messages:
        producer.send(topic_name, message)
        count+=1
      producer.flush()
      print("Wrote {0} messages into topic: {1}".format(count, topic_name))

In [ ]:
write_to_kafka("Traffic", realtime_raw_traffic)
write_to_kafka("News", realtime_raw_news)
write_to_kafka("Weather", realtime_raw_weather)

Wrote 216 messages into topic: Traffic
Wrote 11 messages into topic: News
Wrote 216 messages into topic: Weather


In [ ]:
!kafka_2.12-3.5.0/bin/kafka-topics.sh --bootstrap-server $KAFKA_BOOTSTRAP_SERVERS --list

News
Traffic
Weather


# Spark Streaming

In [ ]:
import pandas as pd
from IPython.display import display, clear_output
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as f
from pyspark.ml import PipelineModel
from pyspark.sql.functions import udf
from pyspark.sql.streaming import DataStreamReader
import html
from pyspark.sql.functions import col, when , concat, lit


In [ ]:
# Extracting information from Kafka topic
traffic_kafkaDF = spark \
    .read \
    .format('kafka') \
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVERS) \
    .option('subscribe', "Traffic") \
    .option("startingOffsets", "earliest") \
    .load()

news_kafkaDF = spark \
    .read \
    .format('kafka') \
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVERS) \
    .option('subscribe', "News") \
    .option("startingOffsets", "earliest") \
    .load()

weather_kafkaDF = spark \
    .read \
    .format('kafka') \
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVERS) \
    .option('subscribe', "Weather") \
    .option("startingOffsets", "earliest") \
    .load()

In [ ]:
traffic_kafkaDF = traffic_kafkaDF.select(col('topic'), col("value").cast("string")).toPandas()
news_kafkaDF = news_kafkaDF.select(col('topic'), col("value").cast("string")).toPandas()
weather_kafkaDF = weather_kafkaDF.select(col('topic'), col("value").cast("string")).toPandas()

# Real-time data preprocessing

In [ ]:
traffic_df, news_df, weather_df = preprocessing_realtime_data(traffic_kafkaDF, weather_kafkaDF, news_kafkaDF)
X = extract_reatime_data(traffic_df, news_df, weather_df)

<ipython-input-29-ab88bd67f77d>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_df[station][i] += news_df['polarity'][j]
<ipython-input-29-ab88bd67f77d>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_df[station][i] = empty_df[station][i] /station_dic[station][date]


In [ ]:
print(X.shape)
print(np.isnan(X).any())

(65, 6048)
False


In [ ]:
PATH = '/content/drive/MyDrive/Phân tích dữ liệu lớn - DS200.N21/Data/Real-time/'
np.savetxt(PATH +"real_time_node_features.txt", X)

# Demo Real-time

In [ ]:
from time import sleep
from IPython.display import display, clear_output

for x in range(0, 100):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"Seconds passed: {x*5}")
        print(f" {'-'*80} Raw Traffic Data {'-'*80}")
        display(convert_dfdict_to_df(traffic_kafkaDF, value = 'value'))
        sleep(2)
        print(f" {'-'*80} Proccessed Traffic Data {'-'*80}")
        display(traffic_df)
        sleep(5)

        print(f" {'-'*80} Raw News Data {'-'*80}")
        display(convert_dfdict_to_df(news_kafkaDF, value = 'value'))
        sleep(2)
        print(f" {'-'*80} Proccessed Traffic Data {'-'*80}")
        display(news_df)
        sleep(10)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break
print("Live view ended...")

Showing live view refreshed every 5 seconds
Seconds passed: 55
 -------------------------------------------------------------------------------- Raw Traffic Data --------------------------------------------------------------------------------


,datetime,01004,02384,02413,03022,100001,10011,19065,22001,22043,...,74229,9658,9827,9834-PR,9836-PR,9849,F3FWY001,T0085,T0231,T0345
0,2022-06-28 00:00:00,155.0,104.0,41.0,41.0,NaN,NaN,549.0,474.0,32.0,...,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-28 01:00:00,126.0,56.0,30.0,34.0,NaN,NaN,341.0,252.0,13.0,...,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-28 02:00:00,106.0,65.0,29.0,20.0,NaN,NaN,298.0,248.0,7.0,...,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-06-28 03:00:00,140.0,65.0,37.0,40.0,NaN,NaN,314.0,328.0,13.0,...,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-06-28 04:00:00,248.0,181.0,101.0,71.0,NaN,NaN,556.0,794.0,18.0,...,154.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,2022-07-06 19:00:00,1387.0,804.0,323.0,455.0,1591.0,1486.0,2572.0,3746.0,509.0,...,1006.0,NaN,466.0,390.0,968.0,NaN,678.0,NaN,NaN,NaN
212,2022-07-06 20:00:00,1013.0,659.0,228.0,396.0,1241.0,1210.0,2327.0,2958.0,323.0,...,757.0,NaN,402.0,299.0,801.0,NaN,551.0,NaN,NaN,NaN
213,2022-07-06 21:00:00,1100.0,592.0,231.0,363.0,1081.0,1110.0,2169.0,3006.0,268.0,...,756.0,NaN,390.0,226.0,593.0,NaN,401.0,NaN,NaN,NaN
214,2022-07-06 22:00:00,1004.0,483.0,160.0,280.0,1007.0,777.0,1827.0,2218.0,162.0,...,555.0,NaN,255.0,97.0,474.0,NaN,345.0,NaN,NaN,NaN


 -------------------------------------------------------------------------------- Proccessed Traffic Data --------------------------------------------------------------------------------


,01004,02384,02413,03022,100001,10011,19065,22001,22043,24014,...,74229,9658,9827,9834-PR,9836-PR,9849,F3FWY001,T0085,T0231,T0345
datetime,,,,,,,,,,,,,,,,,,,,,
2022-06-28 00:00:00,155.0,104.0,41.0,41.0,0.0,0.0,549.0,474.0,32.0,435.0,...,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-06-28 01:00:00,126.0,56.0,30.0,34.0,0.0,0.0,341.0,252.0,13.0,281.0,...,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-06-28 02:00:00,106.0,65.0,29.0,20.0,0.0,0.0,298.0,248.0,7.0,217.0,...,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-06-28 03:00:00,140.0,65.0,37.0,40.0,0.0,0.0,314.0,328.0,13.0,279.0,...,71.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-06-28 04:00:00,248.0,181.0,101.0,71.0,0.0,0.0,556.0,794.0,18.0,563.0,...,154.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-06 19:00:00,1387.0,804.0,323.0,455.0,1591.0,1486.0,2572.0,3746.0,509.0,1785.0,...,1006.0,0.0,466.0,390.0,968.0,0.0,678.0,0.0,0.0,0.0
2022-07-06 20:00:00,1013.0,659.0,228.0,396.0,1241.0,1210.0,2327.0,2958.0,323.0,1461.0,...,757.0,0.0,402.0,299.0,801.0,0.0,551.0,0.0,0.0,0.0
2022-07-06 21:00:00,1100.0,592.0,231.0,363.0,1081.0,1110.0,2169.0,3006.0,268.0,1426.0,...,756.0,0.0,390.0,226.0,593.0,0.0,401.0,0.0,0.0,0.0


 -------------------------------------------------------------------------------- Raw News Data --------------------------------------------------------------------------------


,post_id,text,time,likes,comments,shares,post_url
0,361677882740640,ROYAL NATIONAL PARK:\nAudley Weir is closed be...,02-07-2022 03:12:30,190,7,12,https://facebook.com/livetrafficnsw/posts/3616...
1,361798246061937,FLOODED ROADS IN SYDNEY - UPDATE AT 8:30PM:\n-...,02-07-2022 07:52:25,514,60,139,https://facebook.com/livetrafficnsw/posts/3617...
2,361934472714981,MILPERRA - UPDATE AT 11:50PM: Due to further f...,02-07-2022 12:58:11,230,29,32,https://facebook.com/livetrafficnsw/posts/3619...
3,361967616045000,GEORGES HALL - UPDATE 3:45AM: Henry Lawson Dri...,02-07-2022 14:03:54,246,25,32,https://facebook.com/livetrafficnsw/posts/3619...
4,362252232683205,LATEST ON FLOOD AFFECTED ROADS IN SYDNEY AS OF...,02-07-2022 23:54:57,608,70,107,https://facebook.com/livetrafficnsw/posts/3622...
5,362815415960220,LATEST ON FLOOD AFFECTED ROADS ON MONDAY 4 JUL...,03-07-2022 20:32:15,272,42,80,https://facebook.com/livetrafficnsw/posts/3628...
6,363421322566296,LATEST ON ROADS AFFECTED BY FLOODING AT 12PM:\...,04-07-2022 20:13:46,409,90,134,https://facebook.com/livetrafficnsw/posts/3634...
7,363611919213903,ROADS AFFECTED BY FLOODING - UPDATE ON TUESDAY...,05-07-2022 04:36:59,520,69,171,https://facebook.com/livetrafficnsw/posts/3636...
8,364070192501409,ROADS AFFECTED BY FLOODING - UPDATE ON WEDNESD...,05-07-2022 21:06:12,316,113,76,https://facebook.com/livetrafficnsw/posts/3640...
9,364598869115208,HALFWAY CREEK: FINAL UPDATE AT 5:00PM: All lan...,06-07-2022 17:38:37,941,289,283,https://facebook.com/livetrafficnsw/posts/3645...


break
Live view ended...
